In [12]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, classification_report, f1_score, recall_score
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler

In [2]:
data = pd.read_csv('Data/25day_dataset.csv')

In [3]:
label = []
for i in range(len(data)):
    if data['wahing_machine'].iloc[i]==0 and data['dishwasher'].iloc[i]==0 and data['oven'].iloc[i]==0:
        label.append(0)
    elif data['wahing_machine'].iloc[i]>0:
        label.append(1)
    elif data['dishwasher'].iloc[i]>0:
        label.append(2)
    elif data['oven'].iloc[i]>0:
        label.append(3)
data['Class'] = label

In [4]:
Test_set1 =data[data['DateTime'].str.startswith('2022-01-01')]
Test_set2 =data[data['DateTime'].str.startswith('2022-01-02')]
Test_set3 =data[data['DateTime'].str.startswith('2022-01-03')]
Test_set4 =data[data['DateTime'].str.startswith('2022-01-04')]
Test_set5 =data[data['DateTime'].str.startswith('2022-01-05')]
Test_set6 =data[data['DateTime'].str.startswith('2022-01-06')]
Test_set7 =data[data['DateTime'].str.startswith('2022-01-07')]
Test_set8 =data[(data['DateTime'].str.startswith('2022-01-08')) | (data['DateTime'].str.startswith('2022-01-09'))]
Test_set9 =data[(data['DateTime'].str.startswith('2022-01-10')) | (data['DateTime'].str.startswith('2022-01-11'))]
Test_set10 =data[(data['DateTime'].str.startswith('2022-01-12')) | (data['DateTime'].str.startswith('2022-01-13'))]
test_set_list= [Test_set1, Test_set2, Test_set3, Test_set4, Test_set5, Test_set6, Test_set7, Test_set8, Test_set9, Test_set10]

Crea una copia del dataset e ripuliscilo dai dati presenti nelle fold. Utile per addestramenti esterni. Per addestramenti interni(quindi uno per fold bisogna eliminare solo il Test_set corrente)

In [5]:
dfi = data.copy()
for i in range(len(test_set_list)):
    dfi.drop(test_set_list[i].index, axis=0, inplace=True)

Training del modello esternamente alle fold facendo una compressione stile Binning per le time series. Vengono presi 5 campioni consecutivi e si fa la media dei valori comprimendoli in uno solo. Per la colonna della classe si usa una moda in modo da scegliere quella piu rappresenta fra i 5 campioni.

In [41]:
#Binning
Training_set = dfi.copy()
X = Training_set.groupby(np.arange(len(dfi))//5).mean()
y = Training_set.groupby(np.arange(len(dfi))//5).agg(lambda x:x.value_counts().index[0])
X= X.drop(['DateTime', 'Class', 'wahing_machine', 'dishwasher', 'oven'], axis=1)
y= y.Class
    
#Undersampling per bilanciare il dataset
two= int((y.values == 2).sum())
three= int((y.values == 3).sum())
undersample = RandomUnderSampler(sampling_strategy={0: 15000, 1: 15000, 2: two, 3: three})
X_new, y_new = undersample.fit_resample(X, y)
    
#Modello del classificatore
clf = RandomForestClassifier(class_weight= {0: 5, 1: 5, 2: 1, 3: 1},
                                criterion= 'entropy', 
                                max_depth= 25, 
                                max_features= 'sqrt',
                                max_samples= 0.5,
                                n_estimators= 50)    
clf.fit(X_train, y_train)

#Calcolo f1_score e recall per ogni fold
f_score={}
recall={}
for i in range(len(test_set_list)):
    X_test = test_set_list[i].copy().drop(['DateTime', 'Class', 'wahing_machine', 'dishwasher', 'oven', 'ReactivePower', 'harmonic'], axis=1)
    y_preds = model.predict(X_test)
    y_test = test_set_list[i].Class
    print("CM:\n" + str(confusion_matrix(y_test,y_preds)) + "\n")
    print(classification_report(y_test,y_preds))
    r = recall_score(y_test,y_preds, average= None) 
    f = f1_score(y_test,y_preds, average= None)
    f_score[i] = f
    recall[i] = r 

#Calcolo le medie delle metriche di prestazione
d= pd.DataFrame.from_dict(f_score, orient='index').replace(0, np.nan)
print(d.mean(axis=0, skipna=True))
d= pd.DataFrame.from_dict(recall, orient='index').replace(0, np.nan)
print(d.mean(axis=0, skipna=True))

KeyboardInterrupt: 

Funzione per il calcolo 

In [40]:
f_score={}
recall={}
for i in range(len(test_set_list)):
    
    print("start cleaning")
    dfi = data.drop(test_set_list[i].index)
    y_= dfi.Class
    X_= dfi.drop(['DateTime', 'Class', 'wahing_machine', 'dishwasher', 'oven'], axis=1)
    X = X_.groupby(np.arange(len(dfi))//5).mean()
    y = y_.groupby(np.arange(len(dfi))//5).agg(lambda x:x.value_counts().index[0])
    print("stop cleaning")
       
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    two= int((y_train.values == 2).sum())
    three= int((y_train.values == 3).sum())
    undersample = RandomUnderSampler(sampling_strategy={0: 8000, 1: 8000, 2: two, 3: three})
    X_new, y_new = undersample.fit_resample(X_train, y_train)
    
    
    clf = RandomForestClassifier(class_weight= {0: 5, 1: 1, 2: 1, 3: 1},
                                   criterion= 'entropy', 
                                   max_depth= 20,
                                   max_features= 'sqrt',
                                   max_samples= 0.4,
                                   n_estimators= 25,
                                )
    clf.fit(X_train, y_train)
    y_preds= clf.predict(X_test)
    print("CM:\n" + str(confusion_matrix(y_test,y_preds)) + "\n")
    print("Test_set\n" + str(classification_report(y_test,y_preds)))
    
    X_test = test_set_list[i].copy().drop(['DateTime', 'Class', 'wahing_machine', 'dishwasher', 'oven'], axis=1)
    y_preds = clf.predict(X_test)
    y_test = test_set_list[i].Class
    print("CM:\n" + str(confusion_matrix(y_test,y_preds)) + "\n")
    print(classification_report(y_test,y_preds))
    r = recall_score(y_test,y_preds, average= None) 
    f = f1_score(y_test,y_preds, average= None)
    f_score[i] = f
    recall[i] = r 
    
d= pd.DataFrame.from_dict(f_score, orient='index').replace(0, np.nan)
print(d.mean(axis=0, skipna=True))
d= pd.DataFrame.from_dict(recall, orient='index').replace(0, np.nan)
d.mean(axis=0, skipna=True)
print(d.mean(axis=0, skipna=True))

start cleaning
stop cleaning
CM:
[[116958    334     11      3]
 [   460   3916      0      1]
 [    76      0   1554      1]
 [    64      4      2   1032]]

Test_set
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    117306
           1       0.92      0.89      0.91      4377
           2       0.99      0.95      0.97      1631
           3       1.00      0.94      0.96      1102

    accuracy                           0.99    124416
   macro avg       0.98      0.95      0.96    124416
weighted avg       0.99      0.99      0.99    124416

CM:
[[81580    80     2     3]
 [  317  3736     0     0]
 [    0     0     0     0]
 [  143     0     0   539]]



C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     81665
           1       0.98      0.92      0.95      4053
           2       0.00      0.00      0.00         0
           3       0.99      0.79      0.88       682

    accuracy                           0.99     86400
   macro avg       0.74      0.68      0.71     86400
weighted avg       0.99      0.99      0.99     86400

start cleaning
stop cleaning
CM:
[[116958    337      5      6]
 [   427   3922      0      2]
 [    80      0   1597      0]
 [    68      2      1   1011]]

Test_set
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    117306
           1       0.92      0.90      0.91      4351
           2       1.00      0.95      0.97      1677
           3       0.99      0.93      0.96      1082

    accuracy                           0.99    124416
   macro avg       0.98      0.95      0.96    124416
weighted avg     

C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1

              precision    recall  f1-score   support

           0       0.98      0.99      0.99     78269
           1       0.91      0.81      0.86      7008
           2       0.00      0.00      0.00         0
           3       1.00      0.93      0.96      1123

    accuracy                           0.98     86400
   macro avg       0.72      0.68      0.70     86400
weighted avg       0.98      0.98      0.98     86400

start cleaning
stop cleaning
CM:
[[116710    345      4      7]
 [   524   4214      0      0]
 [    79      0   1419      0]
 [    84      1      0   1029]]

Test_set
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    117066
           1       0.92      0.89      0.91      4738
           2       1.00      0.95      0.97      1498
           3       0.99      0.92      0.96      1114

    accuracy                           0.99    124416
   macro avg       0.98      0.94      0.96    124416
weighted avg     

...

stop cleaning
CM:
[[116967    311     11      5]
 [   472   3955      0      0]
 [    88      0   1619      0]
 [    56      3      0    929]]

Test_set
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    117294
           1       0.93      0.89      0.91      4427
           2       0.99      0.95      0.97      1707
           3       0.99      0.94      0.97       988

    accuracy                           0.99    124416
   macro avg       0.98      0.94      0.96    124416
weighted avg       0.99      0.99      0.99    124416

CM:
[[80661   280     6     9]
 [  572  2685     0     1]
 [    0     0     0     0]
 [  668     1     3  1514]]

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     80956
           1       0.91      0.82      0.86      3258
           2       0.00      0.00      0.00         0
           3       0.99      0.69      0.82      2186

    accuracy              

C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1

stop cleaning
CM:
[[112170    335      9      4]
 [   408   3839      0      1]
 [    51      1   1370      0]
 [    68      3      1    972]]

Test_set
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    112518
           1       0.92      0.90      0.91      4248
           2       0.99      0.96      0.98      1422
           3       0.99      0.93      0.96      1044

    accuracy                           0.99    119232
   macro avg       0.98      0.95      0.96    119232
weighted avg       0.99      0.99      0.99    119232

CM:
[[160375    378    149     15]
 [  1206   6237      1      0]
 [  1244     42   2347      0]
 [   626      0      0    180]]

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    160917
           1       0.94      0.84      0.88      7444
           2       0.94      0.65      0.77      3633
           3       0.92      0.22      0.36       806

    accura

C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1

stop cleaning
CM:
[[112240    298      7      6]
 [   475   3842      0      6]
 [    68      1   1344      1]
 [    82      4      1    857]]

Test_set
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    112551
           1       0.93      0.89      0.91      4323
           2       0.99      0.95      0.97      1414
           3       0.99      0.91      0.94       944

    accuracy                           0.99    119232
   macro avg       0.98      0.94      0.95    119232
weighted avg       0.99      0.99      0.99    119232

CM:
[[159666    184     56     16]
 [  1234   5256      0      0]
 [   721      0   2742      0]
 [   469      1      1   2454]]

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    159922
           1       0.97      0.81      0.88      6490
           2       0.98      0.79      0.88      3463
           3       0.99      0.84      0.91      2925

    accura